## Init



In [0]:
!wget https://github.com/OrtenburgerAdrian/bachelorarbeit-data/archive/master.zip
!unzip master.zip

import re
import string
from google.colab import files

import numpy as np
import os
import subprocess
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras import Sequential
from keras.preprocessing import sequence
from keras.models import load_model
from keras.layers import Embedding, LSTM, Dense, Dropout


In [0]:
!ls ./Thesis-Data-master/

##Löschen doppelter Zeilen 

In [0]:
string = ''
fobj = open("bachelorarbeit-data-master/a_c_k")
lines = fobj.readlines()
lines = sorted(lines)
fobj_out = open("a_c_k","w")
for line in lines:
    if string == line.rstrip():
        print(line.rstrip())
    else:
        fobj_out.write(line)
    string = line.rstrip()
fobj.close()
fobj_out.close()


##erstellen einer ID-Tabelle 

In [0]:
fobj = open("bachelorarbeit-data-master/a_w_k")
list_w=[]
list_z=[]

for line in fobj:
  line = re.sub('\n' , '', line)
  if line in list_w:
    list_z[list_w.index(line)] += 1 
  else:
    list_w.append(line)
    list_z.append(1)
    

fobj_out = open("c_w_t","w")


for x in list_w:
  if list_z[list_w.index(x)]<3:
    i=list_w.index(x)
    list_w[i]=''
    list_z[i]=0
  
i=0   
for x in list_w:
  if x=='':
    continue
  fobj_out.write(str(x)+' '+str(i)+' '+str(list_z[list_w.index(x)])+'\n')
  i+=1
fobj_out.close()
fobj.close() 

In [0]:
!cat c_w_t

## Kommentare mit ID-Tabelle übersätzen

In [0]:
fobj_out = open("c_i_z1","w")
fobj2 = open("bachelorarbeit-data-master/c_w_t")
fobj = open("bachelorarbeit-data-master/a_c_k")
list_w=[]
for w in fobj2:
  list_w.append(w.split(' ')[0])
for line in fobj:
  worter = line.split(';')[1].split(' ')
  x=''
  for wort in worter:
    wort = re.sub('\n' , '', wort)
    wort = wort.lower()
    if wort in list_w:
      if list_w.index(wort)==0:
        index = 3136
      else:
        index = list_w.index(wort)
      x=x+str(index)+' '
  fobj_out.write(str(line.split(';')[0])+';'+ x+'\n')
  
string = ''
fobj = open("c_i_z1")
lines = fobj.readlines()
lines = sorted(lines)
fobj_out = open("c_i_z","w")
for line in lines:
    if line.split(';')[1] != '\n':

        if string == line.rstrip():
            print(line.rstrip())
        else:
            fobj_out.write(line)
    string = line.rstrip()
fobj.close()
fobj_out.close()

In [0]:
!cat c_i_z

##Berechnung der benötigten Kommentare länge

In [19]:
prozent = 90 # so viele Prozent wie gewüscht sind 

fobj = open("bachelorarbeit-data-master/c_i_z")
a = []
i = 0
while i < 1000:
  a.append(0)
  i += 1
for x in fobj:
  a[len( x.split(';')[1].split(' '))]+=1
fobj.close()
i = 0
for x in a:
  i = i + x 
y = 0
u = 0
while u <len(a):
  y = y + a[u]
  if y / i * 100 > prozent:
    print (u)
    break
  u += 1

34


##Übertragung des Sentimentwertes in ein Array 

In [0]:
fobj = open("bachelorarbeit-data-master/c_i_z")
lines = fobj.readlines()
Y=[]
for line in lines:
  if line.split(';')[1] == "\n":
    continue
  if line.split(';')[0] == "0":
    Y.extend([[1, 0]])
  if line.split(';')[0] == "1":
    continue
  if line.split(';')[0] == "2":
    Y.extend([[0, 1]])

fobj.close()
y = np.array(Y, dtype=float)
print(y.shape)

(2485, 2)


##Übertragung des Kommmentars (Zahlenkolonne) in ein Array 

In [0]:
fobj = open("bachelorarbeit-data-master/c_i_z")
list_c=[]
for line in fobj:
  list_w=[]
  line = re.sub('\n' , '', line)
  if line.split(';')[1] == "":
    continue  
  if line.split(';')[0] == "1":
    continue
  words = line.split(';')[1].split(' ')
  for word in words:
    try:
      if int(word) == 0: 
        word= 1316
      list_w.append(int(word))
    except:
      continue
  list_c.append(list_w)
print (len(list_c))

##Aufteillung in Test-, Trainings-, und Validirungsdaten .

In [0]:
max_words = 34
validierungsdaten=50

#aufteilen in Test- und Trainingsdaten
X_train,X_test,y_train,y_test=train_test_split(
    list_c, y,test_size=0.10, random_state = 1)

#Kürzung bzw. Erweiterung der Kommentare auf die maximale Länge.
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

#Aufteilung in Trainings- und Validierungsdaten
X_valid, y_valid = X_train[:validierungsdaten], y_train[:validierungsdaten]
X_train, y_train = X_train[validierungsdaten:], y_train[validierungsdaten:]

## Das LSTM

In [0]:
embedding_size=32
library=3137
max_words= 34

model=Sequential()
model.add(Embedding(library, embedding_size, input_length=max_words))
model.add(LSTM(2))
model.add(Dropout(0.25))
#model.add(Dense(2, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))
print(model.summary())
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

## Training

In [0]:
i=1
while i<200:
  model.fit(X_train, y_train, validation_data=(X_valid, y_valid), 
                                          batch_size=1, epochs=1)
  scores = model.evaluate(X_test, y_test, verbose=0)
  print('Test accuracy:', scores[1])
  model.save('modell_acc.'+str(scores[1])+'_epoch.'+str(i))
  i+=1

In [0]:
!ls

## Download eines Modells

In [0]:
files.download('XXXXX')

## Laden eines Modells 

In [0]:
model = load_model('bachelorarbeit-data-master/modell_acc.0.8634538181335571_epoch.12')

## Auswertung der Ergebnisse

In [0]:

results = model.predict(X_test)
i=0
false_negativ=0
true_negativ=0
false_positv=0
true_positv=0

while i< len(results):
  if y_test[i][0] == 0:
    if results[i][0] > 0.5:
      false_positv += 1
    else:
      true_positv += 1
  if y_test[i][0] == 1:
    if results[i][0] > 0.5:
      true_negativ += 1
    else:
      false_negativ += 1
  i+=1
print("false_negativ")
print(false_negativ)
print("true_negativ")
print(true_negativ)
print("false_positv")
print(false_positv)
print("true_positv")
print(true_positv)

## Rückübersätzer

In [0]:
results = model.predict(X_test)
fobj2 = open("bachelorarbeit-data-master/c_w_t")
fobj = open("bachelorarbeit-data-master/c_i_z")
list_w=[]

for w in fobj2:
  list_w.append(w.split(' ')[0])
i=0  
print('Sentiment; Vorhergesagt: x > 0.5 == positiv; Kommentar')
print('0=negativ; 1=positiv')
while i < len(X_test):
  com = X_test[i]
  comment = '' 
  comment = str(int(y_test[i][1]))+';'+"%f"%results[i][1]+';'
  for word in com:
    if word == 0:
      continue
    if word == 3136:
      word=0
    comment = comment+ ' ' + list_w[word]
  comment = re.sub('; ' , ';', comment)
  print(comment)
  i+=1

# Zusatzfunktionen

## Löschen aller Sonderzeichen, Zahlen und einfache Rechtschreibkorrektur 

In [0]:
spell = SpellChecker('de')
spell.word_frequency.add('XD')

fobj = open("Thesis-Data-master/6k_spellcheck_word-rechtschreib-prüfung.formt")
fobj_out = open("dbComments.formt","w")
i=0
for line in fobj:
  i+=1
  kor_kommentar=''
  
  zeile=line.split(';')
  
  if len(zeile) == 2:
    
    if (zeile[0]=='0') or (zeile[0]=='1') or (zeile[0]=='2'):
      sentiment= zeile[0]
    else:
      continue
    kommentar= zeile[1]
    
    #alle ungewollten Zeichen löschen:
    allow = "ßäöüÄÜÖa-zA-Z "
    kommentar = re.sub('[^%s]' % allow, ' ', kommentar)
    while (kommentar.find('  ')!= -1):
      kommentar = re.sub('  ',' ',kommentar)
    if len(re.sub(' ','',kommentar))<=1:
      continue
    if kommentar[0]==' ':
      kommentar=kommentar[1:]
    
    #Wörter korrigieren
    for wort in kommentar.split(' '):
      wort_m2=''
      misspelled = spell.unknown([wort])
      for wort_m in misspelled:
          if wort_m  != '':
            wort_m2  = spell.correction(wort_m )
            if wort_m2 != wort_m:
              if len(wort_m2) > 1:
                if not wort_m.find('ue'):
                  if wort_m2.find('ue'):
                    wort_m2 = re.sub('ue','ü',wort_m2)
                if not wort_m.find('ae'):
                  if wort_m2.find('ae'):
                    wort_m2 = re.sub('ae','ä',wort_m2)
                if not wort_m.find('oe'):
                  if wort_m2.find('oe'):
                    wort_m2 = re.sub('oe','ö',wort_m2)
                  
                
                print ('correction '+str(i)+'  '+ wort_m + ' >> '+ wort_m2)
                wort=wort_m2
              else:
                print ('unknown '+str(i)+'  '+ wort_m)
                wort=wort_m2
            else:
              print ('unknown '+str(i)+'  '+ wort_m)
              wort=wort_m2
              
      kor_kommentar = kor_kommentar + wort + ' '
      
    while (kor_kommentar.find('  ')!= -1):
      kor_kommentar = re.sub('  ',' ',kor_kommentar)
    if len(re.sub(' ','',kor_kommentar))<=1:
      continue
    fobj_out.write(sentiment+ ';' + kor_kommentar+ '\n' )
    
  else:
    continue

## Kommentare mit FastText übersätzen

In [0]:
!git clone https://github.com/facebookresearch/fastText.git
#Modell downloaden und entpacken
os.chdir("/content/fastText")
!make
!mkdir data
!mkdir result
!mkdir model
os.chdir("/content/fastText/model")
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.de.zip
!unzip wiki.de.zip
os.chdir("/content/fastText")

#Datei zum übersätzen vorbereiten
fobj = open("dbComments.formt")
str2=""
for line in fobj:
  str1=line
  
  for i in str1.split(';'):
    if (i != "0" and i != "1" and i != "2") :
      str2=str2+i
fobj_out = open("dbComments.trans","w")
fobj_out.write(str2)
fobj.close()
fobj_out.close()

#Datei übersätzen 
!./fasttext print-word-vectors model/wiki.de.bin <dbComments.formt >>dbComments.w_vec
